In [2]:
#统计ipv6数据集中 ptr and NXDomain中 查询私有ip域名的数量
import pandas as pd
from tqdm import tqdm
import csv
from easy_fun import judge_priptr,fqdn2pubsuf
file_name='../source_data_new/v6/DNS_COLLECT_LOG.csv'
#下面这个要对应准
fin=open(file_name,'r',encoding='utf-8')
csv_in=csv.reader(fin)
Dic_item={}
cnt123=0
cnt123_pri=0
Dic_item['index']=[]
Dic_item['unixtime']=[]
Dic_item['DiaID']=[]
Dic_item['QorR']=[]
Dic_item['Rcode']=[]
Dic_item['Qname']=[]
Dic_item['Qtype']=[]
Dic_item['Client']=[]
Dic_item['Resolver']=[]
Dic_item['Rjson']=[]
i=0
for line in tqdm(csv_in):
    i+=1
    # if i<4450000:
    #     continue
    if i>10000:
        break
    #这个顺序无所谓
    QorR=line[39]
    if QorR=='0':
        continue
    Qname=line[48].lower()
    Qtype=int(line[46])
    if Qtype!=12:
        continue
    Rcode=line[45]
    if Rcode!='3':
        continue
    cnt123+=1
    # if fqdn2pubsuf(Qname)!='null':
    #     continue
    if judge_priptr(Qname)==True:
        cnt123_pri+=1
        Dic_item['index'].append(i)
        Dic_item['unixtime'].append(line[0])
        Dic_item['DiaID'].append(line[112])
        Dic_item['QorR'].append(line[113])
        Dic_item['Rcode'].append(line[119])
        Dic_item['Qname'].append(line[124])
        Dic_item['Qtype'].append(line[125])
        Dic_item['Client'].append(line[24])
        Dic_item['Resolver'].append(line[33])
        Dic_item['Rjson'].append(line[129])
dfout=pd.DataFrame(Dic_item)
dfout_sorted=dfout.sort_values(by=['Rcode'])
print('sorting dataframe...')
dfout_sorted.to_csv('other_data/check_log123pri6.csv')

ModuleNotFoundError: No module named 'easy_fun'

In [3]:
#抽取指定查询的日志
import data
import pandas as pd
from tqdm import tqdm
import csv
from main import init
from easy_fun import judge_priptr,fqdn2pubsuf
file_name=data.dir_in+'/2023-08-30-DNS.csv'
print('reading source data...')
#下面这个要对应准
fin=open(file_name,'r',encoding='utf-8')
csv_in=csv.reader(fin)
print('extracting data to dataframe...')
#这个决定新的csv里面前后顺序
Dic_item={}
Dic_item['index']=[]
Dic_item['unixtime']=[]
Dic_item['DiaID']=[]
Dic_item['QorR']=[]
Dic_item['Rcode']=[]
Dic_item['Qname']=[]
Dic_item['Qtype']=[]
Dic_item['Client']=[]
Dic_item['Resolver']=[]
Dic_item['Rjson']=[]
init()
cnt_nopubsuf=0
cnt_nopubsuf0=0
cnt_nopubsuf3=0
cnt_nopubsufoth=0
i=0
for line in tqdm(csv_in):
    i+=1
    # if i<4450000:
    #     continue
    # if i>100000:
    #     break
    QorR=line[113]
    if QorR=='0':#对于所有响应
        continue
    Rcode=line[119]
    Qtype=line[125]
    Qname=line[124].lower()
    if fqdn2pubsuf(Qname)=='null':
        cnt_nopubsuf+=1
        if Rcode=='0':
            cnt_nopubsuf0+=1
        elif Rcode=='3':
            cnt_nopubsuf3+=1
        else:
            cnt_nopubsufoth+=1
        Dic_item['index'].append(i)
        Dic_item['unixtime'].append(line[0])
        Dic_item['DiaID'].append(line[112])
        Dic_item['QorR'].append(line[113])
        Dic_item['Rcode'].append(line[119])
        Dic_item['Qname'].append(Qname)
        Dic_item['Qtype'].append(line[125])
        Dic_item['Client'].append(line[24])
        Dic_item['Resolver'].append(line[33])
        Dic_item['Rjson'].append(line[129])
dfout=pd.DataFrame(Dic_item)
dfout_sorted=dfout.sort_values(by=['Rcode','Resolver'])
print('sorting dataframe...')
dfout_sorted.to_csv('other_data/check_logpubsuf.csv')#检查是否为 非公共域名 和 私有内网ptr查询
print(cnt_nopubsuf)
print(cnt_nopubsuf0)
print(cnt_nopubsuf3)
print(cnt_nopubsufoth)

ipv6.kg.qq.com


In [4]:
# 输出A记录的NXDomain的域名排名
from tqdm import tqdm
import csv
file_name='../source_data_new/v6/DNS_COLLECT_LOG.csv'
#下面这个要对应准
fin=open(file_name,'r',encoding='utf-8')
csv_in=csv.reader(fin)
next(csv_in)
Dic_domain_num={}
i=0
for line in tqdm(csv_in):
    i+=1
    # if i>100000:
    #     break
    if line[45]=='3' and line[46]=='1':#Rcode是3 Qtype是1
        Qname=line[48].lower()
        if Qname not in Dic_domain_num:
            Dic_domain_num[Qname]=1
        else:
            Dic_domain_num[Qname]+=1
Dic_domain_num_sorted=dict(sorted(Dic_domain_num.items(),key=lambda x:x[1],reverse=True))
fout=open('../other_data/check_log_ANXdomain_num.csv','w',encoding='utf-8',newline='')
csv_out=csv.writer(fout)
for key,val in Dic_domain_num_sorted.items():
    csv_out.writerow([key,val])

KeyboardInterrupt: 